In [ ]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from dbb.models import Users, Lessons, UserAnswers , UseCases # Adjust based on your actual models
import json

# Connect to the production database
prod_engine = create_engine(os.environ['DATABASE_URI'])
Session = sessionmaker(bind=prod_engine)
prod_session = Session()

# Function to copy data from production to test database
def copy_data_from_prod_to_test():
    # Copy Users
    prod_users = prod_session.query(Users).all()
    for user in prod_users:
        db.session.add(Users(id=user.id, username=user.username, email=user.email,
                            hashed_password=user.hashed_password, use_case_difficulty=1,
                            score=user.score if user.score else 0, is_admin=user.is_admin, lesson_id=user.lesson_id if user.lesson_id else 1))

    # Copy lessons
    prod_lessons = prod_session.query(Lessons).all()
    for lesson in prod_lessons:
        db.session.add(Lessons(id=lesson.id, title=lesson.title))

    # Copy user answers
    prod_user_answers = prod_session.query(UserAnswers).all()
    for user_answer in prod_user_answers:
        db.session.add(UserAnswers(id=user_answer.id, user_id=user_answer.user_id,
                       use_case_id=user_answer.use_case_id, question_id=user_answer.question_id,
                       option_id=user_answer.option_id, is_correct=user_answer.is_correct, created_at=user_answer.created_at,
                       lesson_id=user_answer.lesson_id
        ))

    # Copy use cases
    prod_use_cases = prod_session.query(UseCases).all()
    for use_case in prod_use_cases:
        db.session.add(UseCases(id=use_case.id, description=use_case.description,
                       type=use_case.type,
                        lesson_id=1,
                       created_by_user=use_case.created_by_user, risk_factors=json.dumps(use_case.risk_factors)
                       , difficulty_id=use_case.difficulty_id,
        ))

    db.session.commit()

In [ ]:
import config
from datetime import datetime
from typing import Any, Dict, Optional

import markdown
from flask import Flask
from flask import current_app as app
from flask import jsonify, redirect, render_template, request, session, url_for
from flask_login import (LoginManager, UserMixin, current_user, login_required,
                         login_user, logout_user)

from app import db, create_app
from dbb.models import (DifficultyLevel, Lessons, Options, Questions, UseCases,
                        UserAnswers, UserLessonInteraction, Users)
from config import TestingConfig

app = create_app('TestingConfig')
app_context = app.app_context()
app_context.push()
db.create_all()

# Populate the test database
copy_data_from_prod_to_test()

In [ ]:
def find_similar_use_case(current_use_case_id, user_id):
    """
    Finds a use case similar to the current one based on shared risk factors.

    This function searches for a use case that shares risk factors with the current use case but has not yet been successfully completed by the user. This can help in recommending similar scenarios for further training or assessment.

    Args:
        current_use_case_id (int): The ID of the current use case.
        user_id (int): The ID of the user for whom similar use cases are being searched.

    Returns:
        UseCases | None: Returns a similar UseCase object if found, otherwise None.
    """
    current_use_case = UseCases.query.get(current_use_case_id)
    if not current_use_case:
        return None

    current_risk_factors = current_use_case.risk_factors

    # Query to find a similar use case based on risk factors
    similar_use_case = UseCases.query.filter(
        UseCases.id != current_use_case_id,
        UseCases.risk_factors.contains(current_risk_factors),
        ~UseCases.id.in_(
            db.session.query(UserAnswers.use_case_id)
            .filter(UserAnswers.user_id == user_id, UserAnswers.is_correct)
            .distinct()
        ),
    ).first()

    return similar_use_case

In [ ]:
current_use_case_id = 2
current_use_case = UseCases.query.get(current_use_case_id)
current_risk_factors = current_use_case.risk_factors
current_risk_factors


In [ ]:
json.loads(current_risk_factors)

In [ ]:
json.loads(current_risk_factors)

In [ ]:
db.session.remove()
db.drop_all()
app_context.pop()

In [ ]:
import config
from datetime import datetime
from typing import Any, Dict, Optional

import markdown
from flask import Flask
from flask import current_app as app
from flask import jsonify, redirect, render_template, request, session, url_for
from flask_login import (LoginManager, UserMixin, current_user, login_required,
                         login_user, logout_user)

from app import db, create_app
from dbb.models import (DifficultyLevel, Lessons, Options, Questions, UseCases,
                        UserAnswers, UserLessonInteraction, Users)
from config import DevelopmentConfig

app = create_app(DevelopmentConfig)

In [ ]:
UseCases.query.filter_by(lesson_id=lesson_id)